In [1]:
from breeze_connect import BreezeConnect
from jproperties import Properties
import urllib.parse
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from pyotp import TOTP

In [22]:
config_file_name='app.properties'
configs = Properties()
with open(config_file_name, 'rb') as read_prop:
    configs.load(read_prop,"utf-8")
if len(configs) == 0:
    print("Error while reading properties files")    

In [3]:
url_key = configs.get("URL").data if configs.get("URL") else None
api_key = configs.get("APP_KEY").data if configs.get("APP_KEY") else None
api_secret = configs.get("SECRET_KEY").data if configs.get("SECRET_KEY") else None
totp = TOTP(configs.get("TOTP").data if configs.get("TOTP") else None)
username_key=configs.get("USERNAME").data if configs.get("USERNAME") else None
password_key=configs.get("PASSWORD").data if configs.get("PASSWORD") else None

In [12]:
def auto_login():
    options = webdriver.ChromeOptions() #newly added 
    options.add_argument("--headless=new")
    browser = webdriver.Chrome(options=options)
    browser.get(url_key+urllib.parse.quote_plus(api_key))
    username = browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
    password = browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input')
    username.send_keys(username_key)
    password.send_keys(password_key)

    # Checkbox
    browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
    # click login button
    browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
    time.sleep(2)
    pin = browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')
    token = totp.now()
    pin.send_keys(token)
    browser.find_element(By.XPATH,'/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
    time.sleep(1)
    session_token=browser.current_url.split('apisession=')[1][:8]
    browser.quit()
    return session_token

## run only once in a day
session_token = auto_login()

In [25]:
with open(config_file_name, 'wb') as write_prop:
    configs["SESSION_TOKEN"] = session_token
    configs.store(write_prop, encoding="utf-8")

In [20]:
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret,session_token=session_token)

In [21]:
print(breeze.get_funds())

{'Success': {'bank_account': '913010030505193 ', 'total_bank_balance': 0.0, 'allocated_equity': 0.0, 'allocated_fno': 0.0, 'block_by_trade_equity': 0.0, 'block_by_trade_fno': 0.0, 'block_by_trade_balance': 0.0, 'unallocated_balance': '0'}, 'Status': 200, 'Error': None}


In [15]:
import pandas as pd
getDF = lambda data : pd.DataFrame(data["Success"])
getDF(breeze.get_portfolio_holdings(exchange_code="NSE",
                                from_date="2023-08-01T06:00:00.000Z",
                                to_date="2024-03-09T06:00:00.000Z",
                                stock_code="",
                                portfolio_type=""))

,stock_code,exchange_code,quantity,average_price,booked_profit_loss,current_market_price,change_percentage,answer_flag,product_type,expiry_date,strike_price,right,category_index_per_stock,action,realized_profit,unrealized_profit,open_position_value,portfolio_charges
0,GOLDEX,NSE,59,53,0,55.44,1.31578947368421,N,None,None,None,None,None,None,None,None,None,None
1,NIFBEE,NSE,7,203.49,0,248.67,0.0201110127906041,N,None,None,None,None,None,None,None,None,None,None
2,INDPAI,NSE,5,1525.89,0,1363.55,-0.774996361519429,N,None,None,None,None,None,None,None,None,None,None
3,ITC,NSE,30,446.4,0,413.55,1.3975726369989,N,None,None,None,None,None,None,None,None,None,None
